# Setup

In [3]:
import matplotlib.pyplot as plt
import matplotlib.tri as tri
import numpy as np
import h5py
import copy
import random
import pandas as pd
import keras_tuner as kt
from scipy import stats
from numba import cuda
from sklearn.model_selection import train_test_split as _train_test_split

import tensorflow as tf
from tensorflow.keras import Input
from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow_probability import distributions
from tensorflow_probability import math
from tensorflow_probability import distributions
from tensorflow_probability import math as tfpmath

# workaround to import pdn and CLR from another forlder while they are not installed
import sys
sys.path.insert(0,'../ML_tracer_painting/')
sys.path.insert(0,'./')

#import pdn
import clr

In [4]:
%matplotlib inline

plt.rcParams.update({
    "text.usetex": False,
    "font.family": "sans-serif"
})

In [5]:
directory_path = "/home/draco/work/quijote/Pk_LH/"

# Define couple of useful functions

In [6]:
# Function defining loss plot.

def plot_loss(histories, ylim=None, logy=False):
    for key, history in histories.items():
        plt.plot(
            np.array(range(len(history.history['val_loss'])))-0.5, 
            history.history['loss'], 
            label='loss'
        )
        plt.plot(history.history['val_loss'], label='val_loss')

        if logy:
            plt.semilogy()
        if ylim is not None:
            plt.ylim(ylim[0], ylim[1])
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        print(key)
        plt.title(key)
        plt.grid(True)
        plt.show()

In [7]:
# Split the available dataset in the train validation and test set

def train_val_test_split(
        properties,
        labels,
        val_fraction,
        test_fraction,
        random_state=1,
):
    test_size = round(len(properties) * test_fraction)
    val_size = round(len(properties) * val_fraction)

    train_prop, test_prop, train_lab, test_lab = _train_test_split(
          properties, labels, 
          test_size=test_size, random_state=random_state)

    train_prop, val_prop, train_lab, val_lab = _train_test_split(
          train_prop, train_lab,
          test_size=val_size, random_state=random_state)
    
    return train_prop, val_prop, test_prop, train_lab, val_lab, test_lab

# Load pre-processed data

Load the target variables from file

In [ ]:
filename = directory_path+"Pk_LH/latin_hypercube_params.txt"

labels_van = np.loadtxt(
    filename,
    skiprows=1
)
f = open(filename)
header = f.readline()
f.close()

In [ ]:
label_names_vanilla = [
    "Omega_m",
    "Omega_b",
    "h",
    "n_s",
    "sigma_8",
]

label_LaTeX_names_vanilla = [
    r"$\Omega_m$",
    r"$\Omega_b$",
    r"$h$",
    r"$n_s$",
    r"$\sigma_8$"
]

Target variables are normalized so that they have mean=0 and std=0

In [ ]:
sigmas_van = []
means_van = []

for i in range(len(labels_van[0])):
    s = np.std(labels_van[:,i])
    m = np.mean(labels_van[:,i])
    labels_van[:,i] = (labels_van[:,i] - m)/s
    sigmas_van.append(s)
    means_van.append(m)

In [ ]:
def get_data_dict(
        data,
        first_feature_index,
        val_fraction = 0.2,
        test_fraction = 0.2,
        random_state = 1,
):    
    
    data_dict = {
        "train" : {},
        "val" : {},
        "test" : {},
    }
    
    lbl = data[:,0:first_feature_index].copy()
    ftr = data[:,first_feature_index:].copy()    
    
    data_dict["train"]["ftr"],\
    data_dict["val"]["ftr"],\
    data_dict["test"]["ftr"],\
    data_dict["train"]["lbl"],\
    data_dict["val"]["lbl"],\
    data_dict["test"]["lbl"] = \
    train_val_test_split(
        ftr, lbl,
        val_fraction,
        test_fraction,
        random_state,
    )
    
    return data_dict

In [ ]:
P0 = []
for i in range(2000):
    P0.append(np.loadtxt(directory_path+"Pk_LH/Pk/"+str(i)+"/Pk_m_z=0.txt")[:32,1] ) # The indeces select kmax = 0.2 and [k, Pk]
P0 = np.array(P0)

data_LH_vanilla = {}
data_LH_vanilla["P0"] = np.concatenate(( labels_van, P0 ), axis=1)

for x in data_LH_vanilla.keys():
    data_LH_vanilla[x] = get_data_dict(np.array(data_LH_vanilla[x]), 5)

In [ ]:
plt.plot(P0.T)
plt.loglog()
plt.show()

# Analysis

Create layers

In [ ]:
training_set_properties = data_LH_vanilla["P0"]["train"]["ftr"]

def create_layers(architecture, dropout_rate):

    # the features are normalized to have mean=0 and std=1
    inputs = Input(shape=training_set_properties.shape[1])
    normalize_layer = layers.Normalization()
    normalize_layer.adapt(training_set_properties)
    norm_inputs = normalize_layer(inputs)
    layer = norm_inputs

    # Add 3 hidden dense layers with 128 neuron each
    # Each followed by a dropout layer
    for n_nodes in architecture:
        tlayer = layers.Dense(
            n_nodes,
            activation="selu",
            kernel_initializer='he_normal',
        )(layer)
        layer = layers.Dropout(dropout_rate)(tlayer)

    # Add the output layer combining means and sigmas
    means = layers.Dense(
        5,
        activation="linear",
        kernel_initializer="he_normal",
    )(layer)

    sigmas = layers.Dense(
        5, 
        activation="elu_plus_one",
        kernel_initializer="he_normal",
    )(layer)

    output_layer = layers.Concatenate()([means, sigmas])
    
    return inputs, output_layer

Define loss

In [ ]:
def mse_means_and_sigmas_uncorrelated(y_true, y_pred):
    means_pred, sigmas_pred = tf.split(y_pred, num_or_size_splits=2, axis=1)
    
    y_true = tf.cast(y_true, dtype=y_pred.dtype)
    
    squared_differences = tf.math.square(y_true - means_pred)
    sigmas2_sigma = tf.math.reduce_mean(tf.math.square(squared_differences - tf.math.square(sigmas_pred)), 0)
    sigmas2 = tf.math.reduce_mean(squared_differences, 0) 

    loss = tf.math.reduce_mean(tf.math.log(sigmas2) + tf.math.log(sigmas2_sigma))
    #loss = tf.math.reduce_mean(sigmas2 + sigmas2_sigma)

    return loss

Create model

In [ ]:
def create_model(inputs, output_layer):
    model = Model(inputs=inputs, outputs=output_layer)

    model.compile(
        loss=mse_means_and_sigmas_uncorrelated,
        optimizer=tf.optimizers.Adam(learning_rate=1.e-5),
        #optimizer=tf.optimizers.Adam(learning_rate=1.e-3),
    )

    model.summary()
    
    return model

Define callbacks

In [ ]:
max_lr = 1.e-3
clr_triangular = CLR.CyclicLR(#mode='exp_range',
                              base_lr=max_lr/4.,
                              max_lr=max_lr,
                              step_size=3*4, # recommended (2-8) x (training iterations in epoch)
                              gamma=0.99994)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=30, #The number pavid people that want to do better than 10 but not commit to 100 use.
    restore_best_weights=True,
    verbose=0,
)

Define some ancillary stuff

In [ ]:
histories = {}

In [ ]:
def check_predictions(trueY, predicY, predicE, label='quantity [some units]', numbins=100, title=None):
    fig, ax = plt.subplots(ncols=3,sharex=True,figsize=(9,2.8))#6.4,2.8 #6.4,4.8

    ax[0].errorbar(
        x=trueY[:], y=predicY,
        yerr=predicE,
        elinewidth=0.5,
        linewidth=0,
        #bins='log', xscale='log', yscale='log',
        #gridsize=numbins
    )
    extremes = [np.min([trueY, predicY]),np.max([trueY, predicY])]    
    ax[0].set_xlabel('True '+label)
    ax[0].set_ylabel('Predicted '+label)
    ax[0].plot(extremes, extremes, c='k')
    ax[0].set_xlim(extremes[0], extremes[1])
    ax[0].set_ylim(extremes[0], extremes[1])
    ax[0].set_aspect('equal', adjustable='box')
    
    ymean = np.mean(trueY)
    R2 = 1.-np.sum((trueY-predicY)**2) / np.sum((trueY-ymean)**2)
    
    ax[0].text(0.975, 0.025, r'$R^2$=%.2f'
               "\n"
               r"$\chi^2$=%.2f" %(R2, np.sum((trueY - predicY)**2/predicE**2)/(len(predicE)-2)),
               style='italic', transform=ax[0].transAxes,
        bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 2}, ha="right", va="bottom")
    
    ax[1].plot(trueY, predicE, marker=".", lw=0, markersize=2, alpha=1)
    ax[1].set_xlabel('True '+label)
    ax[1].set_ylabel('Standard deviation')
    ax[1].text(0.975, 0.025, r'$\langle\sigma \rangle$=%.2f'
               "\n"
               r"RMSE=%.2f" %(np.mean(predicE), np.sqrt(np.mean((predicY-trueY)**2))), 
               style='italic', transform=ax[1].transAxes,
               bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 2},
               ha="right", va="bottom")
    
    ax[2].grid(axis="y",alpha=0.5,ls="--")
    
    ax[2].plot(trueY, (predicY-trueY)/predicE, marker=".", lw=0, markersize=2, alpha=1)
    ax[2].set_xlabel('True '+label)
    ax[2].set_ylabel(r'Bias [$\sigma$]')
    ax[2].grid(axis="y",alpha=0.5,ls="--")
    ax[2].text(0.975, 0.025, r"$\langle bias \rangle$=%.2f"
               "\n"
               r"$\langle |bias| \rangle$=%.2f" % (np.mean((predicY-trueY)/predicE), np.mean(np.abs(predicY-trueY)/predicE)), 
               style='italic', transform=ax[2].transAxes,
               bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 2},
               ha="right", va="bottom")
    plt.tight_layout()
    
    if title is not None:
        plt.subplots_adjust(left=0.1, right=0.975, top=0.9, bottom=0.2)
        plt.suptitle(title)#, fontdict={'horizontalalignment': "center"})
    else:
        plt.subplots_adjust(left=0.05, right=0.975, top=0.975, bottom=0.2)
    fig.show()

Fit

In [ ]:
inputs, output_layer = create_layers([32, 32, 32], 0.3)
model = create_model(inputs, output_layer)

In [ ]:
histories["P0"] = model.fit(
    training_set_properties,
    data_LH_vanilla["P0"]["train"]["lbl"],
    validation_data=(data_LH_vanilla["P0"]["train"]["ftr"],
                     data_LH_vanilla["P0"]["train"]["lbl"]),
    batch_size=512,
    epochs=1000000,
    callbacks=[clr_triangular, early_stopping],
    verbose=0,
)

In [ ]:
plot_loss(histories)#, logy=True, ylim=[15, 25])

In [ ]:
data = data_LH_vanilla["P0"]
data_split = "test"
predictions = model.predict(data[data_split]["ftr"], verbose=0)

for j in range(5):
    check_predictions(
        data[data_split]["lbl"][:,j]*sigmas_van[j]+means_van[j],
        predictions[:,j]*sigmas_van[j]+means_van[j], 
        predictions[:,j+5]*sigmas_van[j],
        label=label_LaTeX_names_vanilla[j],
        title="P0")
print()